In [16]:
from src.lib import *
from src.transfomer import *

## Data Prepocessing

In [17]:
with open("./data/Shakespeare.txt",'r') as f:
    text = f.read()

In [18]:
"".join(list(set(text)))

"-ytmvx\nM!A&3cjpsL; g:,bznZXfKEWk.ONr'THQPR?aSIqeUCDwlGFoJhdu$VYBi"

In [20]:
text = text.lower()

In [21]:
text[:30]

'first citizen:\nbefore we proce'

In [22]:
data = re.split('(\W)',text)

In [23]:
chars = sorted(list(set(data)))

In [24]:
vocab_size = len(chars)

In [25]:
vocab_size

11469

In [26]:
str_to_idx = {char:idx for idx,char in enumerate(chars)}
idx_to_str = {idx:char for idx,char in enumerate(chars)}

In [27]:
encode = lambda s :[str_to_idx[i] for i in s]
decode = lambda l : "".join([idx_to_str[i] for i in l ])

In [28]:
text_encoded = encode(data)
text_encoded = torch.tensor(text_encoded)

In [29]:
data[:6]

['first', ' ', 'citizen', ':', '', '\n']

In [31]:
print(decode(encode(data[:20])))

first citizen:
before we proceed any further, hear 


In [71]:
p = 0.9
n = int(p * len(text_encoded))
train_data = text_encoded[0:n]
test_data = text_encoded[n:]

In [72]:
train_data.shape,test_data.shape

(torch.Size([801758]), torch.Size([89085]))

## Embedding

In [73]:
B,T,C = 4,8,2
torch.tril(torch.rand(3,3))

tensor([[0.8518, 0.0000, 0.0000],
        [0.4929, 0.2754, 0.0000],
        [0.5235, 0.4358, 0.4293]])

In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [75]:
train_data[:100]

tensor([ 3823,     2,  1762,     2,     0,    11,     0,     2,     0,     2,
            0,     1,     0,     2,   844,     2, 11064,     2,  7622,     2,
          416,     2,  4180,     2,     0,     7,     0,     2,     0,     2,
         4719,     2,  6153,     2,  9243,     2,     0,     2,     0,     9,
            0,     2,     0,     2,     0,     2,     0,     1,     0,     2,
            0,     2,     0,     1,     0,     2,   294,     2,     0,    11,
            0,     2,     0,     2,     0,     1,     0,     2,  9243,     2,
            0,     7,     0,     2,     0,     2,  9243,     2,     0,     2,
            0,     9,     0,     2,     0,     2,     0,     2,     0,     1,
            0,     2,     0,     2,     0,     1,     0,     2,  3823,     2])

In [76]:
model = Transformer(
    vocab_size=vocab_size,
    embed_size=384,
    heads=12,
    num_layers=12,
    max_len=1000,
    device=device,
    decode_vocab=decode,
    dropout=0.2,
    lr=6e-4
)

In [77]:
print(sum(p.numel() for p in model.parameters()))

62861133


In [78]:
decode(train_data[:10].tolist())

'first citizen :  '

In [79]:
model.fit(train_data[:1000],batch_size=32,block_size=256,n_iter=1000,print_every=20)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\vodin\.conda\envs\pytorchenv\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\vodin\AppData\Local\Temp/ipykernel_5364/328845669.py", line 1, in <module>
    model.fit(train_data[:1000],batch_size=32,block_size=256,n_iter=1000,print_every=20)
  File "c:\Users\vodin\Documents\Transformer\src\transfomer.py", line 46, in fit
    logits = self.forward(src)
  File "c:\Users\vodin\Documents\Transformer\src\transfomer.py", line 28, in forward
    encoder_out = self.encoder(src,mask=None)
  File "c:\Users\vodin\.conda\envs\pytorchenv\lib\site-packages\torch\nn\modules\module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "c:\Users\vodin\Documents\Transformer\src\encoder_decoder.py", line 44, in forward
    out = layer(out,out,out,mask)
  File "c:\Users\vodin\.conda\envs\pytorchenv\lib\site-packages\torch\nn\modules\

TypeError: object of type 'NoneType' has no len()

In [ ]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
# context = train_data[:50].reshape(1,-1).to(device)
model.inference(context, max_token=10)

In [ ]:
model.plot()

In [ ]:
import re